In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from transformers import RobertaTokenizer, RobertaModel, ViTFeatureExtractor, ViTModel
from PIL import Image
import joblib
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import model as model_function
# 定义设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 读取CSV文件
df = pd.read_csv("最终的记录.csv", encoding='utf-8', encoding_errors='ignore')

# 重命名列（确保与之前一致）
df.columns = [
    '商品名称', '一级种类', '二级种类', '图片地址', 'sku', '文本描述', '折扣率',
    '折扣价', '价格', '星级', '销量', '收入', '评论', 'cc-1', 'cc-2',
    'cc-3', 'DRC-1', 'DRC-2', 'DRC-3', 'RCV-1', 'RCV-2',
    'RCV-3', 'RSV-1', 'RSV-2', 'RSV-3'
]


# 转换数据类型
numeric_columns = ['星级', '折扣率', '折扣价', '价格', '销量', '收入']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 重新计算收入（如果需要）
df['收入'] = df['折扣价'] * df['销量']
df['收入'] = np.log10(df['收入'].replace(0, 1))  # 避免log10(0)错误

In [4]:
#RoBERT模型 
X_data_txt = []
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
tokenizer.pad_token = '<pad>'  
model = RobertaModel.from_pretrained('roberta-base')
for i in tqdm(range(2194), desc="提取文本特征特征"):
    tmp = df['一级种类'][i]+df['二级种类'][i]
    inputs = tokenizer(tmp, return_tensors='pt', padding=True, truncation=True)
    outputs = model(**inputs)
    encoded_text = outputs.last_hidden_state.mean(dim=1)
    encoded_text = encoded_text.detach().numpy()
    X_data_txt.append(encoded_text)

X_data_txt = np.array(X_data_txt)
X_data_txt = X_data_txt.reshape(X_data_txt.shape[0],X_data_txt.shape[2])
X_data_txt_new,dim = model_function.pca_process(X_data_txt)

C:\Users\30794\anaconda3\envs\tmp\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
提取文本特征特征: 100%|████████████████████████████████████████████████████████████| 2194/2194 [04:19<00:00,  8.44it/s]


50 0.99657184


In [5]:
import itertools
# 定义神经网络模型类
class RegressionNN(nn.Module):
    def __init__(self, input_size, activation_fn):
        super(RegressionNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        
        # 激活函数
        if activation_fn == 'ReLU':
            self.activation = nn.ReLU()
        elif activation_fn == 'Sigmoid':
            self.activation = nn.Sigmoid()
        elif activation_fn == 'LeakyReLU':
            self.activation = nn.LeakyReLU()
        else:
            raise ValueError(f"Unsupported activation function: {activation_fn}")
    
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

# 定义超参数网格
learning_rates = [0.001, 0.01, 0.1 ]
epochs_list = [100, 200, 500, 1000]
activation_functions = ['ReLU', 'Sigmoid', 'LeakyReLU']

# 生成所有超参数组合
hyperparameter_combinations = list(itertools.product(learning_rates, epochs_list, activation_functions))


In [6]:
X_data_kind = df[['折扣价', '折扣率', '价格']]
all_data = pd.concat([pd.DataFrame(X_data_kind),pd.DataFrame(X_data_txt_new)],axis=1)
X = np.array(all_data)
Y = df['收入']
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(Y, dtype=torch.float32).view(-1, 1)

In [ ]:
import torch.optim as optim
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# 定义随机种子列表
random_states =  [ 42, 23, 15, 34, 18, 32, 47, 27,8, 52]

# 初始化结果存储结构
results = []

# 遍历每个超参数组合
for lr, epochs, activation in hyperparameter_combinations:
    print(f"\n=== Training with Learning Rate: {lr}, Epochs: {epochs}, Activation: {activation} ===")
    
    # 用于存储当前参数组合的所有随机种子实验结果
    combination_result = {
        'learning_rate': lr,
        'epochs': epochs,
        'activation_function': activation,
        'random_state_metrics': []
    }
    
    # 遍历每个随机种子
    for state in random_states:
        # 划分训练集和测试集
        X_train_tensor, X_test_tensor, y_train_tensor, y_test_tensor = train_test_split(
            X_tensor, y_tensor, test_size=0.01, random_state=state
        )
    
        # 初始化模型
        input_size = 53
        model = RegressionNN(input_size, activation_fn=activation)
        
        # 定义损失函数和优化器
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)
        
        # 训练模型
        model.train()
        for epoch in range(epochs):
            optimizer.zero_grad()
            outputs = model(X_train_tensor)
            loss = criterion(outputs, y_train_tensor)
            loss.backward()
            optimizer.step()
        
        # 评估模型
        model.eval()
        with torch.no_grad():
            predictions = model(X_test_tensor)
            loss_test = criterion(predictions, y_test_tensor).item()
        
        # 计算其他评估指标
        y_test_np = 10**y_test_tensor.numpy()
        predictions_np = 10** predictions.numpy()
        mae = mean_absolute_error(y_test_np, predictions_np)
        rmse = np.sqrt(mean_squared_error(y_test_np, predictions_np))
        r2 = r2_score(y_test_np, predictions_np)
        
        # 记录当前随机种子的指标
        combination_result['random_state_metrics'].append({
            'random_state': state,
            'MAE': mae,
            'RMSE': rmse,
            'R²': r2
        })
    
    # 计算该参数组合的平均指标
    metrics_df = pd.DataFrame(combination_result['random_state_metrics'])
    combination_result['avg_MAE'] = metrics_df['MAE'].mean()
    combination_result['avg_RMSE'] = metrics_df['RMSE'].mean()
    combination_result['avg_R²'] = metrics_df['R²'].mean()
    
    # 添加到总结果
    results.append(combination_result)

# 将结果转换为 DataFrame
results_df = pd.DataFrame(results)

# 展示所有实验结果
print("\n=== All Grid Search Results ===")
print(results_df)

# 保存所有实验结果到 CSV 文件
results_df.to_csv("category_grid_search_results.csv", index=False, encoding='utf-8-sig')
print("\n实验结果已保存到 'category_grid_search_results.csv'")

# 选择平均 R² 最高的参数组合
best_result = results_df.loc[results_df['avg_R²'].idxmax()]
print("\n=== Best Hyperparameter Combination ===")
print(best_result)

# 保存最佳参数组合到 CSV 文件
best_result.to_frame().T.to_csv("number_category_best_parameters.csv", index=False, encoding='utf-8-sig')
print("最佳参数组合已保存到 'number_category_parameters.csv'")



=== Training with Learning Rate: 0.001, Epochs: 100, Activation: ReLU ===

=== Training with Learning Rate: 0.001, Epochs: 100, Activation: Sigmoid ===

=== Training with Learning Rate: 0.001, Epochs: 100, Activation: LeakyReLU ===

=== Training with Learning Rate: 0.001, Epochs: 200, Activation: ReLU ===

=== Training with Learning Rate: 0.001, Epochs: 200, Activation: Sigmoid ===

=== Training with Learning Rate: 0.001, Epochs: 200, Activation: LeakyReLU ===

=== Training with Learning Rate: 0.001, Epochs: 500, Activation: ReLU ===

=== Training with Learning Rate: 0.001, Epochs: 500, Activation: Sigmoid ===

=== Training with Learning Rate: 0.001, Epochs: 500, Activation: LeakyReLU ===

=== Training with Learning Rate: 0.001, Epochs: 1000, Activation: ReLU ===

=== Training with Learning Rate: 0.001, Epochs: 1000, Activation: Sigmoid ===

=== Training with Learning Rate: 0.001, Epochs: 1000, Activation: LeakyReLU ===

=== Training with Learning Rate: 0.01, Epochs: 100, Activation: 